In [33]:
# Read from Cosmos DB analytical store into a Spark DataFrame and display 10 rows from the DataFrame
# To select a preferred list of regions in a multi-region Cosmos DB account, add .option("spark.cosmos.preferredRegions", "<Region1>,<Region2>")

df = spark.read\
    .format("cosmos.olap")\
    .option("spark.synapse.linkedService", "CosmosDb1")\
    .option("spark.cosmos.container", "footprint_analytics")\
    .load()


# take a small sample for demo purpose
df_small = df.limit(5)
display(df_small)

StatementMeta(geo, 13, 35, Finished, Available)

SynapseWidget(Synapse.DataFrame, 595df37b-d829-46e7-a4bf-1ed0abdd453d)

In [34]:
# Define function to flatten any struct type column 

from pyspark.sql.functions import col

def flatten_df(nested_df):
    stack = [((), nested_df)]
    columns = []

    while len(stack) > 0:
        parents, df = stack.pop()

        flat_cols = [
            col(".".join(parents + (c[0],))).alias("_".join(parents + (c[0],)))
            for c in df.dtypes
            if c[1][:6] != "struct"
        ]

        nested_cols = [
            c[0]
            for c in df.dtypes
            if c[1][:6] == "struct"
        ]

        columns.extend(flat_cols)

        for nested_col in nested_cols:
            projected_df = df.select(nested_col + ".*")
            stack.append((parents + (nested_col,), projected_df))

    return nested_df.select(columns)

StatementMeta(geo, 13, 36, Finished, Available)

In [35]:
# Call the flatt_df, which will work on geometry column

from pyspark.sql.types import StringType, StructField, StructType
df_flat = flatten_df(df_small)
display(df_flat)

StatementMeta(geo, 13, 37, Finished, Available)

SynapseWidget(Synapse.DataFrame, bd9db473-0888-41fb-a912-63a4b61a420e)

In [36]:
# Create geodataframe - due to aray in aray nesting, some loops are needed

import geopandas as gpd
from shapely.geometry import shape, Polygon
import pyspark.sql.functions as F
import pandas as pd

df_geom = pd.DataFrame ([], columns = ['geometry'])
gdf_geom = gpd.GeoDataFrame(df_geom).set_geometry('geometry')

geom_collection = []
for x in df_flat.collect():
    row_id = x['id']
    geom = x['geometry_coordinates']
    geom_row = []
    for i in range(len(geom)):
        pair = tuple(geom[i][0])
        geom_row.append(pair)
    values_to_add = {'geometry': Polygon(geom_row)}
    row_to_add = pd.Series(values_to_add)
    gdf_geom = gdf_geom.append(row_to_add, ignore_index=True)

gdf_geom

StatementMeta(geo, 13, 38, Finished, Available)

                                            geometry
0  POLYGON ((-158.18279 21.43854, -158.18268 21.4...
1  POLYGON ((-157.99227 21.45864, -157.99218 21.4...
2  POLYGON ((-157.99392 21.47567, -157.99387 21.4...
3  POLYGON ((-158.01757 21.50056, -158.01745 21.5...
4  POLYGON ((-158.02872 21.47663, -158.02872 21.4...

In [37]:
# set crs to WGS84 = 4326  (in lat long)

gdf_geom = gdf_geom.set_crs(epsg=4326)
gdf_geom.crs

StatementMeta(geo, 13, 39, Finished, Available)

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [38]:
# project to US National Atlas Equal Area (epgs = 2136) - in meter

gdf_geom = gdf_geom.to_crs(epsg=2163)
gdf_geom.crs

StatementMeta(geo, 13, 40, Finished, Available)

<Projected CRS: EPSG:2163>
Name: US National Atlas Equal Area
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - onshore and offshore.
- bounds: (167.65, 15.56, -65.69, 74.71)
Coordinate Operation:
- name: US National Atlas Equal Area
- method: Lambert Azimuthal Equal Area (Spherical)
Datum: Not specified (based on Clarke 1866 Authalic Sphere)
- Ellipsoid: Clarke 1866 Authalic Sphere
- Prime Meridian: Greenwich

In [39]:
# derive area for footprints (in meter^2)
gdf_geom.area

StatementMeta(geo, 13, 41, Finished, Available)

0    114.483904
1    198.073906
2     86.995961
3    239.609187
4    169.204369
dtype: float64